In [47]:
# считывает MovieLens
from os import path
import pandas as pd
import numpy as np

data_dir = "/home/ubuntu/data/movielens/ml-latest-small"

def read_csv(filename: str):
    data = pd.read_csv(path.join(data_dir, filename + ".csv"))
    return data

ratings = read_csv("ratings")
movies = read_csv("movies")

In [48]:
# перекодируем id с пробелами в плотные
ratings["movie_id"] = ratings["movieId"].astype("category").cat.codes.copy()
ratings["user_id"] = ratings["userId"].astype("category").cat.codes.copy()
last_movie_id = ratings["movie_id"].max()
last_user_id = ratings["user_id"].max()

In [49]:
# функция, которая красиво печатает информацию о разреженных матрицах
from scipy.sparse import csr_matrix

def sparse_info(sparse_matrix: csr_matrix) -> None:
    print("Размерности матрицы: {}".format(sparse_matrix.shape))
    print("Ненулевых элементов в матрице: {}".format(sparse_matrix.nnz))
    print("Доля ненулевых элементов: {}"
          .format(sparse_matrix.nnz / sparse_matrix.shape[0] / sparse_matrix.shape[1])
    )
    print("Среднее значение ненулевых элементов: {}".format(sparse_matrix.data.mean()))
    print("Максимальное значение ненулевых элементов: {}".format(sparse_matrix.data.max()))
    print("Минимальное значение ненулевых элементов: {}".format(sparse_matrix.data.min()))

In [50]:
user_x_item = ratings[["user_id", "movie_id"]].as_matrix()
mean_rating = ratings["rating"].mean()
user_item_matrix = csr_matrix(
    (
        (ratings["rating"] > mean_rating).tolist(),
        (
            [pair[0] for pair in user_x_item],
            [pair[1] for pair in user_x_item],
        )
    ),
    shape=(last_user_id + 1, last_movie_id + 1),
    dtype=np.float32
)

sparse_info(user_item_matrix)

Размерности матрицы: (671, 9066)
Ненулевых элементов в матрице: 100004
Доля ненулевых элементов: 0.016439141608663475
Среднее значение ненулевых элементов: 0.5156593918800354
Максимальное значение ненулевых элементов: 1.0
Минимальное значение ненулевых элементов: 0.0


In [51]:
# разобьём наблюдения на тестовую и обучающую выборки
np.random.seed(0)
test_indices = np.random.choice(
    range(user_item_matrix.nnz),
    replace=False,
    size=int(user_item_matrix.nnz * 0.2)
).tolist()
train_data = user_item_matrix.copy()
train_data.data[test_indices] = 0
train_data.eliminate_zeros()
print("размер обучающей выборки: {}".format(train_data.nnz))
test_data = user_item_matrix.copy()
test_data.data[:] = 0
test_data.data[test_indices] = user_item_matrix.data[test_indices]
test_data.eliminate_zeros()
print("размер тестовой выборки: {}".format(test_data.nnz))

размер обучающей выборки: 41289
размер тестовой выборки: 10279


In [52]:
from lightfm import LightFM
from lightfm.evaluation import auc_score

# обучаем абы какую модель
model = LightFM(loss="bpr")
model.fit(
    train_data,
    num_threads=4
)

train_mrr = auc_score(model, train_data).mean()
test_mrr = auc_score(model, test_data).mean()
print('ROC AUC: train %.2f, test %.2f.' % (train_mrr, test_mrr))

ROC AUC: train 0.61, test 0.61.


In [53]:
# выберем информацию по жанрам фильмов
movies_genres = ratings[["movieId", "movie_id"]].drop_duplicates().join(
    movies,
    on="movieId",
    rsuffix="codes",
    lsuffix="movies",
    sort=True
).fillna("None")[["movie_id", "genres"]]
movies_genres["genres_set"] = movies_genres["genres"].apply(lambda x: set(x.split("|")))
movies_genres.head()

,movie_id,genres,genres_set
495,0,Adventure|Children|Fantasy,"{Adventure, Fantasy, Children}"
963,1,Comedy|Romance,"{Comedy, Romance}"
351,2,Comedy|Drama|Romance,"{Comedy, Romance, Drama}"
3108,3,Comedy,{Comedy}
964,4,Action|Crime|Thriller,"{Action, Thriller, Crime}"


In [54]:
# все возможные жанры
from functools import reduce

reduce(lambda acc, ele: acc.union(ele), movies_genres["genres_set"].tolist(), set())

{'(no genres listed)',
 'Action',
 'Adventure',
 'Animation',
 'Children',
 'Comedy',
 'Crime',
 'Documentary',
 'Drama',
 'Fantasy',
 'Film-Noir',
 'Horror',
 'IMAX',
 'Musical',
 'Mystery',
 'None',
 'Romance',
 'Sci-Fi',
 'Thriller',
 'War',
 'Western'}

In [55]:
from sklearn.feature_extraction.text import CountVectorizer

# приравняем None и (no genres listed)
movies_genres.loc[movies_genres["genres"] == "(no genres listed)", "genres"] = "None"
# уберём все спецсимволы, кроме |
movies_genres["genres"] = movies_genres["genres"].apply(
    lambda x: x.replace("-", "")
)

movies_features = CountVectorizer().fit_transform(movies_genres["genres"])
movies_features

<9066x20 sparse matrix of type '<class 'numpy.int64'>'
	with 15786 stored elements in Compressed Sparse Row format>

In [56]:
print(movies_genres["genres"][:1])
print(movies_features[0].todense())

495    Adventure|Children|Fantasy
Name: genres, dtype: object
[[0 1 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0]]


In [57]:
# добавляем к movie_id ещё и информацию о жанрах
from scipy.sparse import hstack, identity

features = hstack([
    identity(movies_genres.shape[0]),
    movies_features
])

In [58]:
# обучаем модель с жанрами
hybrid = LightFM(loss="bpr")
hybrid.fit(
    train_data,
    num_threads=4,
    item_features=features
)

train_mrr = auc_score(hybrid, train_data, item_features=features).mean()
test_mrr = auc_score(hybrid, test_data, item_features=features).mean()
print('ROC AUC: train %.2f, test %.2f.' % (train_mrr, test_mrr))

ROC AUC: train 0.71, test 0.69.
